In [20]:

from google.colab import drive
drive.mount('/content/drive')

import os

BASE_DIR = "/content/drive/MyDrive/MSSP6070"
DATA_DIR = os.path.join(BASE_DIR, "data")
NB_DIR   = os.path.join(BASE_DIR, "notebooks")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(NB_DIR,   exist_ok=True)

print("Base folder:", BASE_DIR)
print("Put parsed.csv in:", DATA_DIR)
print("Notebooks go in:", NB_DIR)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Base folder: /content/drive/MyDrive/MSSP6070
Put parsed.csv in: /content/drive/MyDrive/MSSP6070/data
Notebooks go in: /content/drive/MyDrive/MSSP6070/notebooks


In [21]:
import pandas as pd
import numpy as np
import re
from IPython.display import display


csv_path = f"{DATA_DIR}/parsed.csv"

df = pd.read_csv(csv_path)
df.columns = [c.strip().lower() for c in df.columns]
cols = set(df.columns)


if 'mag' not in cols and 'magnitude' in cols:
    df['mag'] = pd.to_numeric(df['magnitude'], errors='coerce')
else:
    df['mag'] = pd.to_numeric(df.get('mag'), errors='coerce')

# Column names
magtype_col  = 'magtype' if 'magtype' in cols else ('mag_type' if 'mag_type' in cols else None)
tsunami_col  = 'tsunami' if 'tsunami' in cols else None
place_col    = 'place' if 'place' in cols else None
country_col  = 'country' if 'country' in cols else None
state_col    = 'state' if 'state' in cols else None

print("Rows:", len(df))
print("Columns:", sorted(df.columns))

# --- Helper functions (always return bools) ---
def _has(word, text):
    """True if `word` appears as a whole word in `text` (case-insensitive)."""
    if not isinstance(text, str):
        return False
    return bool(re.search(rf"\b{re.escape(word)}\b", text, re.IGNORECASE))

def is_country(row, name):
    if country_col and isinstance(row.get(country_col), str):
        return row[country_col].strip().lower() == name.lower()
    return _has(name, row.get(place_col))

def is_state(row, name):
    if state_col and isinstance(row.get(state_col), str):
        return row[state_col].strip().lower() == name.lower()
    return _has(name, row.get(place_col))

def is_japan(row):     return is_country(row, 'japan')
def is_indonesia(row): return is_country(row, 'indonesia')
def is_nevada(row):    return is_state(row, 'nevada') or _has('nevada', row.get(place_col))


Rows: 9332
Columns: ['alert', 'cdi', 'code', 'detail', 'dmin', 'felt', 'gap', 'ids', 'mag', 'magtype', 'mmi', 'net', 'nst', 'parsed_place', 'place', 'rms', 'sig', 'sources', 'status', 'time', 'title', 'tsunami', 'type', 'types', 'tz', 'updated', 'url']


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
#a: 95th percentile magnitude in Japan (magType='mb')
if magtype_col is not None:
    mask_japan_mb = df.apply(is_japan, axis=1) & (df[magtype_col].astype(str).str.lower() == 'mb')
    mag_95_japan_mb = df.loc[mask_japan_mb, 'mag'].quantile(0.95)
else:
    # If the dataset has no magType, compute for all Japan records
    mask_japan_mb = df.apply(is_japan, axis=1)
    mag_95_japan_mb = df.loc[mask_japan_mb, 'mag'].quantile(0.95)

print("2a) 95th percentile magnitude (Japan, magType='mb'):",
      None if pd.isna(mag_95_japan_mb) else round(float(mag_95_japan_mb), 3))


2a) 95th percentile magnitude (Japan, magType='mb'): 4.9


In [23]:
#(b) % of Indonesia quakes that had tsunamis
if tsunami_col is None:
    pct_indo_tsunami = np.nan
    total_indo = np.nan
    tsunami_indo = np.nan
    print("2b) No 'tsunami' column in this dataset.")
else:
    mask_indo = df.apply(is_indonesia, axis=1)
    total_indo = int(mask_indo.sum())
    tsunami_indo = int((df.loc[mask_indo, tsunami_col].fillna(0).astype(float) > 0).sum())
    pct_indo_tsunami = (tsunami_indo / total_indo * 100.0) if total_indo > 0 else np.nan

    print("2b) Indonesia quakes:", total_indo)
    print("    Indonesia quakes w/ tsunami:", tsunami_indo)
    print("    % with tsunami:", None if pd.isna(pct_indo_tsunami) else round(float(pct_indo_tsunami), 2))


2b) Indonesia quakes: 147
    Indonesia quakes w/ tsunami: 34
    % with tsunami: 23.13


In [24]:
#c) Summary statistics for Nevada
mask_nv = df.apply(is_nevada, axis=1)
nv = df.loc[mask_nv]

print("2c) Nevada summary (describe):")
display(nv.describe(include='all'))

num_summary = nv.select_dtypes(include=[np.number]).describe().T
display(num_summary)


2c) Nevada summary (describe):


,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magtype,...,status,time,title,tsunami,type,types,tz,updated,url,parsed_place
count,0,15.000000,677,677,677.000000,15.000000,677.000000,677,677.000000,677,...,677,6.770000e+02,677,677.0,677,677,677.0,6.770000e+02,677,677
unique,0,NaN,677,677,NaN,NaN,NaN,677,NaN,1,...,2,NaN,626,NaN,2,6,NaN,NaN,677,1
top,NaN,NaN,00657031,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,",nn00657031,",NaN,ml,...,reviewed,NaN,"M -0.3 - 55km ENE of Beatty, Nevada",NaN,earthquake,",geoserve,origin,phase-data,",NaN,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,Nevada
freq,NaN,NaN,1,1,NaN,NaN,NaN,1,NaN,677,...,671,NaN,4,NaN,646,651,NaN,NaN,1,677
mean,NaN,2.440000,NaN,NaN,0.166982,2.400000,154.029527,NaN,0.491728,NaN,...,NaN,1.538314e+12,NaN,0.0,NaN,NaN,-480.0,1.538402e+12,NaN,NaN
std,NaN,0.501142,NaN,NaN,0.166400,4.626013,68.769713,NaN,0.689560,NaN,...,NaN,5.954070e+08,NaN,0.0,NaN,NaN,0.0,6.000267e+08,NaN,NaN
min,NaN,2.000000,NaN,NaN,0.001000,1.000000,29.140000,NaN,-0.500000,NaN,...,NaN,1.537247e+12,NaN,0.0,NaN,NaN,-480.0,1.537307e+12,NaN,NaN
25%,NaN,2.000000,NaN,NaN,0.054000,1.000000,97.670000,NaN,-0.100000,NaN,...,NaN,1.537854e+12,NaN,0.0,NaN,NaN,-480.0,1.537928e+12,NaN,NaN
50%,NaN,2.200000,NaN,NaN,0.113000,1.000000,149.550000,NaN,0.400000,NaN,...,NaN,1.538280e+12,NaN,0.0,NaN,NaN,-480.0,1.538428e+12,NaN,NaN
75%,NaN,2.900000,NaN,NaN,0.234000,1.000000,200.470000,NaN,0.900000,NaN,...,NaN,1.538821e+12,NaN,0.0,NaN,NaN,-480.0,1.538878e+12,NaN,NaN


,count,mean,std,min,25%,50%,75%,max
cdi,15.0,2.440000e+00,5.011416e-01,2.000000e+00,2.000000e+00,2.200000e+00,2.900000e+00,3.300000e+00
dmin,677.0,1.669823e-01,1.664003e-01,1.000000e-03,5.400000e-02,1.130000e-01,2.340000e-01,1.414000e+00
felt,15.0,2.400000e+00,4.626013e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.900000e+01
gap,677.0,1.540295e+02,6.876971e+01,2.914000e+01,9.767000e+01,1.495500e+02,2.004700e+02,3.559100e+02
mag,677.0,4.917282e-01,6.895604e-01,-5.000000e-01,-1.000000e-01,4.000000e-01,9.000000e-01,2.900000e+00
mmi,1.0,2.840000e+00,NaN,2.840000e+00,2.840000e+00,2.840000e+00,2.840000e+00,2.840000e+00
nst,677.0,1.260857e+01,9.890620e+00,3.000000e+00,6.000000e+00,9.000000e+00,1.600000e+01,6.100000e+01
rms,677.0,1.519092e-01,8.474153e-02,5.000000e-04,1.069000e-01,1.463000e-01,1.867000e-01,8.634000e-01
sig,677.0,1.068833e+01,1.925273e+01,0.000000e+00,0.000000e+00,2.000000e+00,1.200000e+01,1.290000e+02
time,677.0,1.538314e+12,5.954070e+08,1.537247e+12,1.537854e+12,1.538280e+12,1.538821e+12,1.539461e+12


In [25]:
#d: Add ring_of_fire column
ring_terms = [
    r'\bbolivia\b',
    r'\bchile\b',
    r'\becuador\b',
    r'\bperu\b',
    r'\bcosta\s*rica\b',
    r'\bguatemala\b',
    r'(?<!new\s)\bmexico\b',   # Mexico but not "New Mexico"
    r'\bjapan\b',
    r'\bphilippines?\b',
    r'\bindonesia\b',
    r'\bnew\s*zealand\b',
    r'\bantarctic\w*',         # Antarctica / Antarctic
    r'\bcanada\b',
    r'\bfiji\b',
    r'\balaska\b',
    r'\bwashington\b',
    r'\bcalifornia\b',
    r'\brussia\b',
    r'\btaiwan\b',
    r'\btonga\b',
    r'\bkermadec\s+islands?\b'
]
ring_regex = re.compile("|".join(ring_terms), re.IGNORECASE)

def on_ring_of_fire(row):
    pieces = []
    for k in ['country', 'state', 'place']:
        if k in df.columns and isinstance(row.get(k), str):
            pieces.append(row[k])
    return bool(ring_regex.search(" | ".join(pieces)))

df['ring_of_fire'] = df.apply(on_ring_of_fire, axis=1)
print("2d) Added column: ring_of_fire")
df[['place','country' if 'country' in df.columns else 'mag', 'ring_of_fire']].head()


2d) Added column: ring_of_fire


,place,mag,ring_of_fire
0,"9km NE of Aguanga, CA",1.35,False
1,"9km NE of Aguanga, CA",1.29,False
2,"8km NE of Aguanga, CA",3.42,False
3,"9km NE of Aguanga, CA",0.44,False
4,"10km NW of Avenal, CA",2.16,False


In [26]:
#(e) Counts on/off Ring of Fire
counts = df['ring_of_fire'].value_counts(dropna=False)
in_ring  = int(counts.get(True, 0))
out_ring = int(counts.get(False, 0))

print("2e) In Ring of Fire:", in_ring)
print("    Outside Ring of Fire:", out_ring)


2e) In Ring of Fire: 4426
    Outside Ring of Fire: 4906


In [27]:
#f:Tsunami count along the Ring of Fire
if tsunami_col is None:
    print("2f) No 'tsunami' column in this dataset.")
else:
    tsunami_on_ring = int((df.loc[df['ring_of_fire'] == True, tsunami_col].fillna(0).astype(float) > 0).sum())
    print("2f) Tsunami count along the Ring of Fire:", tsunami_on_ring)


2f) Tsunami count along the Ring of Fire: 43
